
## SVM for Classification

In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
from libsvm.svmutil import *
# prepare my own photo
# pathlist = glob.glob(r"PIE/myself/*.jpg")
# for i,path in enumerate(pathlist):
#
#     img = cv2.imread(path,0)
#     img = cv2.resize(img,[32,32])
#     print(img.shape)
#     savepath = "PIE/0/"+str(i+1)+".jpg"
#     print(savepath)
#     cv2.imwrite(savepath, img)

# load data

indexlist = np.arange(1,69,1)
np.random.shuffle(indexlist)
indexlist = indexlist[0:25]
#indexlist = np.append(indexlist,0)
indexlist.sort()
print(indexlist)
eachclass_num = [len([f for f in os.listdir(os.path.join('PIE', str(s))) if f.endswith('.jpg')]) for s in indexlist]
total_num = sum(eachclass_num)
vector = np.zeros([total_num,32,32])
label = np.zeros([total_num,1])


n=0
for i,cls in enumerate(indexlist):
    for img_path in [f for f in os.listdir(os.path.join('PIE', str(cls))) if f.endswith('.jpg')]:
        img_path = "PIE/"+str(cls)+"/"+img_path
        img_cur = cv2.imread(img_path,0)
        vector[n] = img_cur
        label[n] = cls
        n += 1

vector = np.reshape(vector,(vector.shape[0],-1))
vector = np.concatenate([vector,label],axis=1)
np.random.shuffle(vector)
label[:,:] = vector[:,1024:1025]
vector = vector[:,0:1024]



self_vector = np.zeros([10,32,32])
self_label = np.zeros([10,1])
n=0
for img_path in [f for f in os.listdir(os.path.join('PIE', str(0))) if f.endswith('.jpg')]:
    img_path = "PIE/" + str(0) + "/" + img_path
    img_cur = cv2.imread(img_path, 0)
    self_vector[n] = img_cur
    n += 1
self_vector = np.reshape(self_vector,(self_vector.shape[0],-1))



trainset = vector[0:int(total_num*0.7)]
trainlabel = label[0:int(total_num*0.7)]
testset = vector[int(total_num*0.7):]
testlabel = label[int(total_num*0.7):]


trainset = np.concatenate([trainset,self_vector[0:7]])
trainlabel = np.concatenate([trainlabel,self_label[0:7]])
# testset = np.concatenate([testset,self_vector[7:]])
# testlabel = np.concatenate([testlabel,self_label[7:]])

print(trainset.shape)
print(trainlabel.shape)
print(testset.shape)
print(testlabel.shape)


def PCA(trainset,k):
    M = trainset - trainset.mean(axis=0)
    train_cov = np.cov(M.T)
    eigenvalues, eigenvectors = np.linalg.eig(train_cov)
    eigenvectors = eigenvectors.real
    sort = eigenvalues.argsort()[::-1]
    eigenvalues = eigenvalues[sort]
    eigenvectors = eigenvectors[:, sort]
    eigenvalues = eigenvalues[0:k]
    eigenfaces = eigenvectors[:,0:k]
    return eigenvalues, eigenfaces

def SVM(trainset,trainlabel,testset,testlabel,cost):
    y = trainlabel.T[0]
    x = trainset
    prob  = svm_problem(y, x)
    config = '-t 0 -b 1' + ' -c '+str(cost)
    param = svm_parameter(config)
    m = svm_train(prob, param)
    y = testlabel.T[0]
    x = testset
    p_label, p_acc, p_val = svm_predict(y, x, m,'-b 1')
    return p_acc[0]

[ 4  5  6 10 11 12 14 15 17 21 22 27 28 29 32 36 41 45 47 48 52 59 64 66
 68]
(2982, 1024)
(2982, 1)
(1275, 1024)
(1275, 1)


In [2]:
acc1 = []
acc1.append(SVM(trainset,trainlabel,testset,testlabel,1))
acc1.append(SVM(trainset,trainlabel,testset,testlabel,0.1))
acc1.append(SVM(trainset,trainlabel,testset,testlabel,0.01))
print("Classification Accuracy (raw images C=1):",acc1[0])
print("Classification Accuracy (raw images C=0.1):",acc1[1])
print("Classification Accuracy (raw images C=0.01):",acc1[2])

Accuracy = 98.7451% (1259/1275) (classification)
Accuracy = 98.7451% (1259/1275) (classification)
Accuracy = 98.6667% (1258/1275) (classification)
Classification Accuracy (raw images C=1): 98.74509803921569
Classification Accuracy (raw images C=0.1): 98.74509803921569
Classification Accuracy (raw images C=0.01): 98.66666666666667


In [4]:
e_num = 200
eigenvalues, eigenfaces = PCA(trainset,e_num)
train_redu = np.dot(trainset, eigenfaces)
test_redu = np.dot(testset, eigenfaces)
acc3 = []
acc3.append(SVM(train_redu,trainlabel,test_redu,testlabel,1))
acc3.append(SVM(train_redu,trainlabel,test_redu,testlabel,0.1))
acc3.append(SVM(train_redu,trainlabel,test_redu,testlabel,0.01))
print("Classification Accuracy (200 dimensions C=1):",acc3[0])
print("Classification Accuracy (200 dimensions C=0.1):",acc3[1])
print("Classification Accuracy (200 dimensions C=0.01):",acc3[2])

Accuracy = 98.5882% (1257/1275) (classification)
Accuracy = 98.5882% (1257/1275) (classification)
Accuracy = 98.5882% (1257/1275) (classification)
Classification Accuracy (200 dimensions C=1): 98.58823529411764
Classification Accuracy (200 dimensions C=0.1): 98.58823529411764
Classification Accuracy (200 dimensions C=0.01): 98.58823529411764


In [5]:
e_num = 80
eigenvalues, eigenfaces = PCA(trainset,e_num)
train_redu = np.dot(trainset, eigenfaces)
test_redu = np.dot(testset, eigenfaces)
acc2 = []
acc2.append(SVM(train_redu,trainlabel,test_redu,testlabel,1))
acc2.append(SVM(train_redu,trainlabel,test_redu,testlabel,0.1))
acc2.append(SVM(train_redu,trainlabel,test_redu,testlabel,0.01))
print("Classification Accuracy (80 dimensions C=1):",acc2[0])
print("Classification Accuracy (80 dimensions C=0.1):",acc2[1])
print("Classification Accuracy (80 dimensions C=0.01):",acc2[2])

Accuracy = 98.2745% (1253/1275) (classification)
Accuracy = 98.3529% (1254/1275) (classification)
Accuracy = 98.5098% (1256/1275) (classification)
Classification Accuracy (80 dimensions C=1): 98.27450980392157
Classification Accuracy (80 dimensions C=0.1): 98.3529411764706
Classification Accuracy (80 dimensions C=0.01): 98.50980392156863
